In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import os
from manim import *

In [ ]:
# Caminho absoluto da raiz do projeto
project_root = Path.cwd().parents[1]  # se o notebook está em /notebooks
print("Diretório atual:", Path.cwd())
os.chdir(project_root)

from src.modules.data_processing.db_reader import DbReader
from src.analysis.championship.plotter import *
from src.modules.animation.manim_constructors import LineChampionshipChart
from src.analysis.data_viz.plotter import *
db = DbReader()

In [ ]:
# Caminhos para os arquivos
pace_path = 'data/features/pace_features.csv'
perf_path = 'data/features/performance_features.csv'
rel_path = 'data/features/reliability_features.csv'
exp_path = 'data/features/experience_features.csv'

# Raw
df_laps = db.run_query_file("data/db_queries/lap_times_report.sql")
df_results = db.run_query_file("data/db_queries/race_results_report.sql")
df_qualify = db.run_query_file("data/db_queries/qualify_report.sql")

# Features
df_pace = pd.read_csv(pace_path)
df_perf = pd.read_csv(perf_path)
df_rel = pd.read_csv(rel_path)
df_exp = pd.read_csv(exp_path)

# Pace features validation and exploration

In [ ]:
df_pace

In [ ]:
df_pace_valid = df_pace.dropna(subset=['avg_pace_vs_field', 'avg_pace_vs_teammate'])

In [ ]:
df_pace_valid.groupby(['year']).count()

Ao olhar o dataframe cru e ver vários N/As, imaginei que teria problemas e que o dataset estaria incompleto. Mas na verdade, ao filtrar e ver os números agrupados acima, até parece que faz sentido, dado que temos uma agregação de dados por ano e por piloto. E, olhando por cima, os números parecem fazer sentido.

## Entendendo dados ausentes

De toda forma, pra mim não está ainda 100% claro os dados que não estão presentes no dataset. Vou tentar investigar isso abaixo.

*Ao trabalhar nesta análise, notei alguns ajustes que deveriam ser feitos, portanto, a análise dos dados ausentes acabou ficando mais curta do que o esperado porque identifiquei erros no pipeline que estavam aumentando a quanitadade de dados ausentes.*

In [ ]:
df_pace_invalid = df_pace[df_pace.isnull().any(axis=1)]

In [ ]:
df_pace_invalid.sort_values(by='year')

In [ ]:
df_pace_invalid.groupby(['year']).count()

Os dados faltantes presentes no dataset agora são referentes à anos que não possuem dados de tempo de volta, ou então porque o piloto não se qualificou pra ter seus dados calculados (terminou pelo menos 2 GPs).

## Explorando os dados de 2025:

In [ ]:
df_pace_2025 = df_pace_valid[df_pace_valid['year'] == 2025]

In [ ]:
for column in df_pace_2025.columns:
    if column not in ['driver_id', 'driver_full_name', 'year', 'driver_surname', 'constructor_name']:
        
        fig, ax = plt.subplots(figsize=(10, 6))

        df_data = df_pace_2025[['driver_surname', column]].sort_values(by=column, ascending=False)
        
        sns.barplot(data=df_data, x='driver_surname', y=column, ax=ax)

        ax.tick_params(axis='x', rotation=45)

        plt.title(column)
        plt.show()

Olhando por cima, os dados parecem todos fazer sentido! Quando eu for fazer o notebook da análise, vou mergulhar em cada um deles, fornecendo uma explicação mais detalhada de cada uma das métricas.

# Performance Features

In [ ]:
df_perf

In [ ]:
df_perf.groupby(['year']).count()

In [ ]:
df_perf[df_perf['year'] >= 1994]

Inicialmente, achei estranho esse dataset ter tantas linhas a mais do que o de pace, mas isso se dá simplesmente porque ele tem dados desde 1950, enquanto o de pace não tem dados anteriores a 1996.

## Explorando os dados de 2025:

In [ ]:
df_perf_2025 = df_perf[df_perf['year'] == 2025]
df_perf_2025

In [ ]:
for column in df_perf_2025.columns:
    if column not in ['driver_id', 'driver_full_name', 'year', 'driver_surname', 'constructor_name']:
        
        fig, ax = plt.subplots(figsize=(10, 6))

        df_data = df_perf_2025[['driver_surname', column]].sort_values(by=column, ascending=False)
        
        sns.barplot(data=df_data, x='driver_surname', y=column, ax=ax)

        ax.tick_params(axis='x', rotation=45)

        plt.title(column)
        plt.show()

Maravilhosamente, os dados acima também parecem fazer bastante sentido! Depois, assim como os acima, serão mais aprofundados no notebook da análise pro vídeo.

# Reliability Features

In [ ]:
df_rel

In [ ]:
df_rel_2025 = df_rel[df_rel['year'] == 2025]

In [ ]:
df_rel_2025

In [ ]:
df_results[df_results['year'] == 2024].groupby(['race_status']).agg({'race_name':'count'}).reset_index()

**ATENÇÃO:** Infelizmente acho que não vai dar pra usar essas features, porque todos os DNFs estão marcados como Mechanical DNF, ou seja, não conseguimos identificar os DNFs por acidente ou erro do piloto. **Ou então usar com a ressalva de que tem esse problema.**

# Experience Features

In [ ]:
df_exp

In [ ]:
df_exp_2025 = df_exp[df_exp['year'] == 2025]

In [ ]:
df_exp_2025